In [1]:
%cd ../

/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/tracking-GNN


/home1/giorgian/anaconda3/envs/jupyter/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from dataclasses import replace
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import os.path
import sys
import logging
import pickle
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from models.agnn_trigger import GNNGraphClassifier
from icecream import ic
from numpy.linalg import inv
import sklearn.metrics as metrics
from datasets import get_data_loaders
from tqdm.notebook import tqdm
from itertools import islice
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

In [3]:
DEVICE = 'cuda:0'

In [4]:
# create model and load checkpoint
model_result_folder = '/disks/disk1/giorgian/old-trainings/agnn/agnn-lr0.00043640124559864087-b512-d128-ReLU-gi5-ln-True-n1000000/experiment_2024-03-24_23:16:30'
model_results_folder = '/disks/disk1/giorgian/old-trainings/trigger_results/trigger_results/agnn/agnn-lr0.0021918383450689467-b512-d16-ReLU-gi3-ln-True-n120000/experiment_2025-04-25_05:03:21 '
#model_results_folder = '../tracking_results/agnn/agnn-lr0.0013362149625589887-b12-d64-ReLU-gi1-ln-True-n40000/experiment_2025-02-02_11:59:41/'

config_file = model_result_folder + '/config.pkl'
config = pickle.load(open(config_file, 'rb'))
data_config = config.get('data')
dphi_max, dz_max = data_config['phi_slope_max'], data_config['z0_max']

model_config = config.get('model', {})
model_config.pop('loss_func')
model_config.pop('name')
model = GNNGraphClassifier(**model_config).to(DEVICE)

def load_checkpoint(checkpoint_file, model, optimizer=None):
    checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model'])
    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer
    return model

# load_checkpoint
checkpoint_dir = os.path.join(model_result_folder, 'checkpoints')
checkpoint_file = sorted([os.path.join(checkpoint_dir, f) for f in os.listdir(checkpoint_dir) if f.startswith('model_checkpoint')])
checkpoint_file = checkpoint_file[-1]
print(checkpoint_file)
model = load_checkpoint(checkpoint_file, model)
print('Successfully reloaded!')

/disks/disk1/giorgian/old-trainings/agnn/agnn-lr0.00043640124559864087-b512-d128-ReLU-gi5-ln-True-n1000000/experiment_2024-03-24_23:16:30/checkpoints/model_checkpoint_011.pth.tar
Successfully reloaded!


/tmp/ipykernel_3435976/1230462323.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))


In [5]:
data_config

{'batch_size': 512,
 'input_dir': '/ssd1/giorgian/hits-data-august-2022-ctypes/trigger/1/',
 'input_dir2': '/ssd1/giorgian/hits-data-august-2022-ctypes/nontrigger/0/',
 'load_full_event': False,
 'n_folders': 2,
 'n_train': 500000,
 'n_valid': 100000,
 'n_workers': 16,
 'name': 'hit_graph_trigger',
 'phi_slope_max': 0.012193355583173944,
 'real_weight': 1,
 'use_cylindrical_std': False,
 'use_hit_type': False,
 'z0_max': 14.220353082111805}

In [6]:
data_config['batch_size'] = 1
data_config['name'] = 'hit_graph_trigger_real_data'
data_config['load_full_event'] = False
data_config['input_dir'] = '/home1/giorgian/real-data-parser/events/'
data_config['input_dir2'] = None
data_config['n_folders'] = 1
data_config['n_train'] = 72
data_config['n_valid'] = 1
data_config['drop_trigger_node'] = True
train_data_loader, valid_data_loader = get_data_loaders(distributed=False, rank=0, n_ranks=0, **data_config)

In [7]:
config['trainer']

{'name': 'gnn_trigger'}

In [64]:
preds = []
for batch in train_data_loader:
    pred = model(batch.to(DEVICE))
    preds.append(pred.detach().cpu().numpy())

preds_trigger = np.array(preds)

In [58]:
np.mean(preds > 0)*100

np.float64(19.444444444444446)

In [11]:
batch.trigger_node

tensor([ True,  True,  True, False, False, False, False, False, False, False,
        False, False, False, False, False,  True,  True,  True,  True,  True,
         True,  True,  True, False, False, False, False, False, False, False,
        False,  True,  True,  True,  True], device='cuda:0')

In [12]:
batch

DataBatch(x=[35, 5], edge_index=[2, 79], trigger=[1], trigger_node=[35], i=[1], filename=[1], batch=[35], ptr=[2])

In [65]:
preds = []
for batch in train_data_loader:
    if torch.all(batch.trigger_node):
        continue
    batch = batch.to(DEVICE)
    batch.x = batch.x[~batch.trigger_node]
    batch.batch = batch.batch[~batch.trigger_node]
    new_indices = torch.arange(torch.sum(~batch.trigger_node)).to(DEVICE)

        
    old_indices = -1 * torch.ones(batch.trigger_node.shape[0]).to(torch.long).to(DEVICE)
    old_indices[~batch.trigger_node] = new_indices
    start = old_indices[batch.edge_index[0]]
    end = old_indices[batch.edge_index[1]]
    new_edge_index = torch.stack([start, end], dim=0)
    new_edge_index = new_edge_index[:, ~torch.any(new_edge_index == -1, dim=0)]
    batch.edge_index = new_edge_index 
    pred = model(batch.to(DEVICE))
    preds.append(pred.detach().cpu().numpy())

preds_nontrigger = np.array(preds)

In [61]:
np.mean(preds > 0)*100

np.float64(8.823529411764707)